# Example mascaret

## Geometry file

In [ ]:
import os.path
from data_manip.formats.mascaretgeo_file import MascaretGeoFile

# File with a single reach
mascgeo_file = MascaretGeoFile(os.path.join(os.environ['HOMETEL'],
                                            'examples', 'mascaret', 'Test6', 'geometrie'),
                               'geo')
print(mascgeo_file.summary())

# File with multiples reaches
mascgeo_file = MascaretGeoFile(os.path.join(os.environ['HOMETEL'],
                                            'examples', 'mascaret', 'Test15', 'mascaret0.geo'))
print(mascgeo_file.summary())

## Opthyca result file

Run some validation mascaret test cases such as `Test18` and `Test26` to have generate opt files ( respectively `sarap_ecr.opt` and `rezo_ecr.opt`).

In [ ]:
%%bash
# Run Test18 : ressaut
cd $HOMETEL/examples/mascaret/Test18 && mascaret.py sarap.xcas

In [ ]:
%%bash
# Run Test26 : confluence
cd $HOMETEL/examples/mascaret/Test26 && mascaret.py rezo.xcas

In [ ]:
import os.path
from data_manip.formats.mascaret_file import MascaretFile


masc_file = MascaretFile(os.path.join(os.environ['HOMETEL'], 'examples', 'mascaret', 'Test26', 'rezo_ecr.opt'))
print(masc_file.summary())

In [ ]:
# Get reaches
masc_file.reaches

In [ ]:
# Get sections of reach `1`
masc_file.reaches[1].sections

In [ ]:
# Get section `1` of reach `1`
masc_file.reaches[1].sections[1]

## Longitudinal profile

In [ ]:
import matplotlib.pyplot as plt
import os.path

from data_manip.formats.mascaret_file import MascaretFile
from postel.plot1d import plot1d


# Selection to display
masc_file = MascaretFile(os.path.join(os.environ['HOMETEL'], 'examples', 'mascaret', 'Test18', 'sarap_ecr.opt'))
varnames = ['Cote de l eau', 'Cote du fond']
reach_id = 1  # first reach
time_index = -1  # last frame
reach = masc_file.reaches[reach_id]

fig, ax = plt.subplots(figsize=(10,8))

for varname in varnames:
    var_pos = masc_file.get_position_var(varname)
    values = masc_file.get_values_at_reach(time_index, reach_id, var_pos)
    plot1d(ax, reach.get_section_pk_list(), values,
           plot_label='%s' % varname,
           x_label='Distance (m)',
           y_label='Cote (m)')

# Displaying legend
ax.legend()

# Showing figure
plt.show()

del plt
del masc_file

## Temporal profile

In [ ]:
import matplotlib.pyplot as plt
import os.path

from data_manip.formats.mascaret_file import MascaretFile
from postel.plot1d import plot1d


# Selection to display
masc_file = MascaretFile(os.path.join(os.environ['HOMETEL'], 'examples', 'mascaret', 'Test26', 'rezo_ecr.opt'))
varname = 'Cote de l eau'
reach_id = 2  # first reach
reach = masc_file.reaches[reach_id]
sections = reach[:5]  # First 5 sections of requested reach

# Build plot
fig, ax = plt.subplots(figsize=(10, 8))

var_pos = masc_file.get_position_var(varname)
for section in sections:
    values = masc_file.get_series(reach_id, section.id, [var_pos])
    plot1d(ax, masc_file.times, values,
           plot_label='%s' % section.name,
           x_label='Temps (s)',
           y_label='Z (m)')

# Displaying legend
ax.legend()

# Showing figure
plt.show()

del plt
del masc_file

## creation .lig file (LIDOP format)

Write a LIDOP format file for compute reprise

In [ ]:
import os.path
from data_manip.formats.mascaret_file import MascaretFile


masc_file = MascaretFile(os.path.join(os.environ['HOMETEL'], 'examples', 'mascaret', 'Test26', 'rezo_ecr.opt'))
# the last time_step for .lig file
id_time = masc_file.ntimestep - 1
# id_time = -1 # give the same result
masc_file.export_as_lig('rezo_ecr.lig', id_time)

Display the .lig file:

In [ ]:
%%bash
cat rezo_ecr.lig

## Creation opthyca file (OPTHYCA format)

This could be interesting if you reduce the variable number or the time step number in the mascaret results file

In [ ]:
import os.path
from data_manip.formats.mascaret_file import MascaretFile

path_file = os.path.join(os.environ['HOMETEL'], 'examples', 'mascaret', 'Test26')

masc_file = MascaretFile(os.path.join(path_file, 'rezo_ecr.opt'))

# List of time index 
times_indexes=[0,-1]
# [initialisation time step, last time step]
vars_indexes=[]
# get position with variable name
vars_indexes.append(masc_file.get_position_var("Cote du fond"))
vars_indexes.append(masc_file.get_position_var("Cote de l eau"))
# get position with abbreviation of variable name
vars_indexes.append(masc_file.get_position_var_abbr("Q"))
print('vars_indexes = ',vars_indexes)
print('times_indexes = ',times_indexes)

# Write a new file
masc_file.write_optfile(os.path.join(path_file, 'rezo_ecr_ZQ.opt'), times_indexes, vars_indexes)

del masc_file

Check the new file which is created.

In [ ]:
# check new file 
masc_file_new = MascaretFile(os.path.join(path_file, 'rezo_ecr_ZQ.opt'))
print(masc_file_new.summary())
del masc_file_new

In [ ]:
%%bash
cat $HOMETEL/examples/mascaret/Test26/rezo_ecr_ZQ.opt